In [20]:
#Seq2seq learning. In this case for audio samples. But can be generalized to text with proper encodings. I used the 
#image for these cases, but what's in a name ? :D

import sys
import os
import cv2
import numpy as np
import tensorflow
import tensorflow as tf
import keras
import keras, keras.layers as L
sys.path.append("..")
from skimage.color import rgb2gray
import keras
from keras.models import load_model
from keras.layers import Input, Dense,Conv2D,RepeatVector, LSTM, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Bidirectional
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np

import os
import cv2

import numpy as np
import pandas as pd
from scipy import signal
from scipy.io import wavfile
import matplotlib.pyplot as plt




In [21]:
import numpy as np
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    sounds = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".wav")]
        for f in file_names:
            fs, dat = (wavfile.read(f))
            sounds.append((dat))
            labels.append(d)
    return sounds, labels

In [22]:
X,y = load_data('speech_commands_v0.01/')

In [23]:
X = np.array(X)

In [24]:
import sklearn
from sklearn.model_selection import train_test_split


In [25]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, random_state = 1)
NUM_CLASSES = 31

In [26]:
Xtest=np.array(([cv2.resize(image,  (16000,1)) for image in Xtest]))
Xtrain=np.array(([cv2.resize(image, (16000,1)) for image in Xtrain]))

In [27]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D,ReLU,Dense,BatchNormalization,Dropout,Activation,MaxPooling1D
from keras.layers.advanced_activations import LeakyReLU

In [28]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

#Convert y labels to integers
le = LabelEncoder()
ytrain2 = le.fit_transform(ytrain)
ytest2 = le.fit_transform(ytest)

In [29]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras_self_attention import SeqSelfAttention

In [30]:
model = Sequential()
model.add( LSTM( 512, input_shape=(1, 16000), return_sequences=True))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))
model.add(Dense(256))
model.add(BatchNormalization())
model.add( LSTM( 512, return_sequences=False))
model.add(Dense(31))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
history = model.fit( Xtrain, ytrain2, batch_size=100, epochs=20, shuffle = False)

Epoch 1/20
  200/51781 [..............................] - ETA: 2:32:49 - loss: 3.4493